# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [2]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing

 Could not import win32api and/or win32con


In [5]:
# set the parameters to run the script
directories = ['/home/elea/Documents/HORAO/repositories/processingMM/tests/data']
calib_directory = '/home/elea/Documents/HORAO/repositories/processingMM/tests/calib'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
parameter_set = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = True

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'pddn'

# do not specify unless you want to use a custom path for the PDDN models
PDDN_models_path = None

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550, 600]

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz

# NB: processing time without PDDN
# 'no_viz': 0.71s
# 'default', save_pdf_figs False: 2.25s
# 'default', save_pdf_figs True: 3.60s
# 'full', save_pdf_figs False: 3.95s
# 'full', save_pdf_figs True: 8.07s
processing_mode = 'default'
save_pdf_figs = False

# define if the wavelenghts should be aligned before processing - and used for the computation
align_wls = True

parameters = batch_processing.get_parameters(directories, calib_directory, wavelengths, parameter_set = parameter_set, 
                                PDDN_mode = PDDN_mode, PDDN_models_path = PDDN_models_path, 
                                processing_mode = processing_mode, run_all = run_all, 
                                save_pdf_figs = save_pdf_figs, align_wls = align_wls)

In [12]:
%%time
times = batch_processing.batch_process_master(parameters)
times

PDDN_models_path: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model
Loading PDDN models...
/home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_550_Fresh_HB.pt
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_550_Fresh_HB.pt -- ...
 [wrn] PDDN Model Summary NOT Available!
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_550_Fresh_HB.pt -- Complete
/home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_600_Fresh_HB.pt
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_600_Fresh_HB.pt -- ...
 [wrn] PDDN Model Summary NOT Available!
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_600_Fresh_HB.pt -- Complete
Loading PDDN m

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 3802.63it/s]


Denoising inference done.

Aligning wavelengths for [550, 600] ...
Aligning wavelenght: 600nm


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 8981.38it/s]


Aligning wavelengths done.




  0%|                                                     | 0/1 [00:00<?, ?it/s]

Processing: /home/elea/Documents/HORAO/repositories/processingMM/tests/data/2022-09-01_T_HORAO-38-BF_FR_S1_1


/home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/multi_img/multi_img_processing.py:277: UserWarning: No calibration was found for the exact date, the one used was 88 day(s) ago.
  incorrect_date(min_duration_distance)
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.84s/it]

Processing with PDDN done.
CPU times: user 4.44 s, sys: 272 ms, total: 4.71 s
Wall time: 2.54 s


{'MM_processing': 0.3247947692871094,
 'azimuth_std_processing': 0.006725788116455078,
 'denoising': 0,
 'save_npz': 0.10157108306884766,
 'full_processing': 0.8184819221496582,
 'plotting': 0.8844499588012695,
 'moving': 0.02387380599975586,
 'total': 1.8494226932525635}

In [28]:
%%time
from processingmm.addons import visualization_lines

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
times = visualization_lines.batch_visualization(parameters)
times

CPU times: user 14.5 s, sys: 57.6 ms, total: 14.6 s
Wall time: 14.6 s


14.573182344436646

In [3]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths
directories = ['/media/elea/ssd/NPP/TU/tocheck']

PDDN_mode = 'both'
run_all = True
align_wavelengths.align_wavelenghts(directories, PDDN_mode, run_all)

Aligning wavelenght: 600nm


  0%|                                                     | 0/1 [00:00<?, ?it/s]

True
 
 >> Exported X3D .cod file as: /media/elea/ssd/NPP/TU/tocheck/2023-05-16_T_HORAO-101-BF_FR_S1_1/raw_data/600nm/600_Intensite_aligned.cod


100%|█████████████████████████████████████████████| 1/1 [00:09<00:00,  9.15s/it]


In [39]:
# add the calibration matrices (A.cod and W.cod) to the folder for processing
# needed by Christopher
from processingmm.addons import add_calibration
add_calibration.add_calibration(directories)

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 15.59it/s]

No calibration files in 600nm
